In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import data_utils
import model_exe

In [10]:
data = data_utils.get_data(source="russell", size=500, start="2020-04-01", interval='1h')

Using 500 symbols
Changing symbol BF.A to BF-A
[*********************100%***********************]  500 of 500 completed


In [23]:
model_exe.run_classifier(data)

Number Train Sequences: 386	total data: 937339
Number Test Sequences: 67	total data: 163614

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_84 (Dense)             (None, 64)                704       
_________________________________________________________________
dense_85 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_86 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_87 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_88 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_89 (Dense)             (None, 1)                 65        
Total params: 17,409
Train